In [ ]:
print("Hello World!")

In [ ]:
! cd ~
! git clone https://github.com/tensorflow/models.git

In [ ]:
! pip install contextlib2
import os
new_python_path = (os.environ.get("PYTHONPATH") or '') + ":models/research/slim"
%env PYTHONPATH=$new_python_path

In [ ]:
! python models/research/slim/download_and_convert_data.py \
--dataset_name=visualwakewords \
--dataset_dir=data/visualwakewords \
--foreground_class_of_interest='car'

In [ ]:
!pip install tf_slim

In [ ]:
! python models/research/slim/train_image_classifier.py \
--train_dir=vww_96_grayscale \
--dataset_name=visualwakewords \
--dataset_split_name=train \
--dataset_dir=data/visualwakewords \
--model_name=mobilenet_v1_025 \
--preprocessing_name=mobilenet_v1 \
--train_image_size=96 \
--use_grayscale=True \
--save_summaries_secs=300 \
--learning_rate=0.045 \
--label_smoothing=0.1 \
--learning_rate_decay_factor=0.98 \
--num_epochs_per_decay=2.5 \
--moving_average_decay=0.9999 \
--batch_size=96 \
--max_number_of_steps=1000000 \
--num_clones=1

2020-09-09 17:33:03.997322: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
Please switch to tf.train.create_global_step
W0909 17:33:13.738971 139943756662144 deprecation.py:323] From models/research/slim/train_image_classifier.py:430: create_global_step (from tf_slim.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0909 17:33:13.788946 139943756662144 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tf_slim/data/parallel_reader.py:249: string_input_producer (from tensorflow.python.training.input) is deprecated and will be r

In [ ]:
! python models/research/slim/eval_image_classifier.py \
--alsologtostderr \
--checkpoint_path=vww_96_grayscale/model.ckpt-1000000 \
--dataset_dir=data/visualwakewords \
--dataset_name=visualwakewords \
--dataset_split_name=val \
--model_name=mobilenet_v1_025 \
--preprocessing_name=mobilenet_v1 \
--use_grayscale=True \
--train_image_size=96

In [ ]:
! python models/research/slim/export_inference_graph.py \
--alsologtostderr \
--dataset_name=visualwakewords \
--model_name=mobilenet_v1_025 \
--image_size=96 \
--use_grayscale=True \
--output_file=vww_96_grayscale_graph.pb

In [ ]:
!git clone -b r1.15 --single-branch https://github.com/tensorflow/tensorflow.git tensorflow
! python tensorflow/tensorflow/python/tools/freeze_graph.py \
--input_graph=vww_96_grayscale_graph.pb \
--input_checkpoint=vww_96_grayscale/model.ckpt-1000000 \
--input_binary=true --output_graph=vww_96_grayscale_frozen.pb \
--output_node_names=MobilenetV1/Predictions/Reshape_1

In [ ]:
! python mobilenet_tflite.py